In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import matplotlib.pyplot as plt
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from lightgbm import log_evaluation

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cafa-5-protein-function-prediction/sample_submission.tsv
/kaggle/input/cafa-5-protein-function-prediction/IA.txt
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta
/kaggle/input/cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
/kaggle/input/cafa-5-protein-function-prediction/Train/go-basic.obo


# Analisando o banco de dados

In [10]:
def read_fasta_file(file_path):
    entries = []
    with open(file_path, "r") as fasta_file:
        lines = fasta_file.readlines()
        i = 0
        while i < len(lines):
            if lines[i].startswith(">"):
                entry = {}
                entry['EntryID'] = lines[i][1:].split()[0]
                
                #entry["OX"] = lines[i].split("OX=")[1].split()[0]
                i += 1
                sequence_lines = []
                while i < len(lines) and not lines[i].startswith(">"):
                    sequence_lines.append(lines[i].strip())
                    i += 1
                entry['seq'] = "".join(sequence_lines)
                entries.append(entry)
            else:
                i += 1
    df = pd.DataFrame(entries)
    df.set_index('EntryID', inplace=True)
    return df

In [11]:
# Entrada
trainFasta = read_fasta_file('/kaggle/input/cafa-5-protein-function-prediction/Train/train_sequences.fasta')

# Saída
trainTermsID = pd.read_csv('/kaggle/input/cafa-5-protein-function-prediction/Train/train_terms.tsv',
                           sep='\t')
# Teste
testFasta = read_fasta_file('/kaggle/input/cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta')

print(trainTermsID['term'].value_counts(normalize=True)*100)
print(trainTermsID['term'].nunique())

GO:0005575    1.732184
GO:0008150    1.719097
GO:0110165    1.701870
GO:0003674    1.466052
GO:0005622    1.319665
                ...   
GO:0031772    0.000019
GO:0042324    0.000019
GO:0031771    0.000019
GO:0051041    0.000019
GO:0102628    0.000019
Name: term, Length: 31466, dtype: float64
31466


In [12]:
trainTermsID['aspect'].value_counts()

BPO    3497732
CCO    1196017
MFO     670114
Name: aspect, dtype: int64

In [13]:
trainFasta

,seq
EntryID,
P20536,MNSVTVSHAPYTITYHDDWEPVMSQLVEFYNEVASWLLRDETSPIP...
O73864,MTEYRNFLLLFITSLSVIYPCTGISWLGLTINGSSVGWNQTHHCKL...
O95231,MRLSSSPPRGPQQLSSFGSVDWLSQSSCSGPTHTPRPADFSLGSLP...
A0A0B4J1F4,MGGEAGADGPRGRVKSLGLVFEDESKGCYSSGETVAGHVLLEAAEP...
P54366,MVETNSPPAGYTLKRSPSDLGEQQQPPRQISRSPGNTAAYHLTTAM...
...,...
A0A286YAI0,METEVDDFPGKASIFSQVNPLYSNNMKLCEAERYDFQHSEPKTMKS...
A0A1D5NUC4,MSAAASAEMIETPPVLNFEEIDYKEIEVEEVVGRGAFGVVCKAKWR...
Q5RGB0,MADKGPILTSVIIFYLSIGAAIFQILEEPNLNSAVDDYKNKTNNLL...


In [14]:
testFasta

,seq
EntryID,
Q9CQV8,MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLL...
P62259,MDDREDLVYQAKLAEQAERYDEMVESMKKVAGMDVELTVEERNLLS...
P68510,MGDREQLLQRARLAEQAERYDDMASAMKAVTELNEPLSNEDRNLLS...
P61982,MVDREQLVQKARLAEQAERYDDMAAAMKNVTELNEPLSNEERNLLS...
O70456,MERASLIQKAKLAEQAERYEDMAAFMKSAVEKGEELSCEERNLLSV...
...,...
P08380,GNCKCDDEGPNVRTAPLTGYVDLGYCNEGWEKCASYYSPIAECCRKKK
C0HK72,RGICLEPKVVGPCKARIRRFYYDSETGKCTPFIYGGCGGNGNNFET...
C0HK73,GSICLEPKVVGPCKAGIRRFYFDSETGKCTLFLYGGCKGNGNNFET...


In [15]:
trainTermsID

,EntryID,term,aspect
0,A0A009IHW8,GO:0008152,BPO
1,A0A009IHW8,GO:0034655,BPO
2,A0A009IHW8,GO:0072523,BPO
3,A0A009IHW8,GO:0044270,BPO
4,A0A009IHW8,GO:0006753,BPO
...,...,...,...
5363858,X5L565,GO:0050649,MFO
5363859,X5L565,GO:0016491,MFO
5363860,X5M5N0,GO:0005515,MFO
5363861,X5M5N0,GO:0005488,MFO


In [16]:
trainTermsID['EntryID'].value_counts().mean()

37.70835735275509

In [17]:
trainFull = trainTermsID.merge(trainFasta, on='EntryID', how='left')
trainFull.drop('aspect', axis=1, inplace=True)
trainFull

,EntryID,term,seq
0,A0A009IHW8,GO:0008152,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,A0A009IHW8,GO:0034655,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
2,A0A009IHW8,GO:0072523,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
3,A0A009IHW8,GO:0044270,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
4,A0A009IHW8,GO:0006753,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
...,...,...,...
5363858,X5L565,GO:0050649,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363859,X5L565,GO:0016491,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363860,X5M5N0,GO:0005515,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...
5363861,X5M5N0,GO:0005488,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...


In [18]:
def keep_most_frequent_outputs(df, percentage):
    # Calculate output frequency
    output_counts = df['term'].value_counts()

    # Calculate the number of classes to keep based on the percentage
    n = int(len(output_counts) * percentage)

    # Get the "n" most frequent outputs
    most_frequent_outputs = output_counts.head(n).index

    # Filter the DataFrame to keep rows with the most frequent outputs
    filtered_df = df[df['term'].isin(most_frequent_outputs)]

    # Calculate the percentage of the database removed
    removed_percentage = (1 - len(filtered_df) / len(df)) * 100
    print("Percentage of database removed: {:.2f}%".format(removed_percentage))
    
    return filtered_df

trainFullD = keep_most_frequent_outputs(trainFull, 0.1)
trainFullD.drop('EntryID', axis=1, inplace=True)

Percentage of database removed: 9.85%


/tmp/ipykernel_34/347505860.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainFullD.drop('EntryID', axis=1, inplace=True)


In [19]:
trainFullD

,term,seq
0,GO:0008152,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
1,GO:0034655,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
2,GO:0072523,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
3,GO:0044270,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
4,GO:0006753,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...
...,...,...
5363857,GO:0003674,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363859,GO:0016491,MDLIPSFSTETWLLLAISLVLLYLYGTYTHGIFRKLGIPGPTPLPF...
5363860,GO:0005515,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...
5363861,GO:0005488,MPDSITNGGRPPAPPSSVSSTTASTTGNFGTRRRLVNRIKKVDELH...


In [20]:
gc.collect()
trainFullenc = pd.get_dummies(trainFullD, columns=['term'], sparse=True)
gc.collect()

def process_dataframe(df):
    df = df.set_index('seq')  # Set 'seq' column as the index
    unique_seqs = df.index.unique()
    unique_classes = df.columns  # Assuming class columns start from index 0
    new_df = pd.DataFrame(0, index=unique_seqs, columns=unique_classes)

    progress_bar = tqdm(total=len(unique_classes), desc="Processing Columns")

    for col in unique_classes:
        gc.collect()
        class_indices = df[df[col] == 1].index
        new_df.loc[class_indices, col] = 1

        progress_bar.update(1)

    progress_bar.close()
    return new_df

trainFullenc = process_dataframe(trainFullenc).reset_index()
trainFullenc

Processing Columns: 100%|██████████| 3146/3146 [36:15<00:00,  1.45it/s] 


,seq,term_GO:0000003,term_GO:0000018,term_GO:0000041,term_GO:0000070,term_GO:0000075,term_GO:0000077,term_GO:0000082,term_GO:0000096,term_GO:0000118,...,term_GO:2001141,term_GO:2001233,term_GO:2001234,term_GO:2001235,term_GO:2001236,term_GO:2001242,term_GO:2001243,term_GO:2001251,term_GO:2001252,term_GO:2001257
0,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MHSTIVYACLLALAVFVALHGTPLAALAENGEGTTQPDYDNSTDYY...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MDRDAEEGRPLSLVNRRPSISAPISGRKSAPASAAAAVAAAAAAAA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MSTIKLLIIGQLWLSIGLISGDDSLDTREGVDLVLKCRFTEHYDST...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138919,MAQYDVLIVGAGHGGAQAAVALRQNKFEGTIAIVGDEPELPYERPP...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138920,FKLLAASGLTRCGRGGLVVTETAVKIVKFHNRTFTLGPVNLKVASE...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138921,MPKGGCPKAPQQEELPLSSDMVEKQTGKKDKDKVSLTKTPKLERGD...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138922,MNKKTKLIHGGHTTDDYTGAVTTPIYQTSTYLQDDIGDLRQGYEYS...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
del trainFullD, trainFull, trainTermsID
gc.collect()

0

# Modelo

In [22]:
# Copiando para processar
trainC = trainFullenc.copy()
del trainFullenc
gc.collect()
trainC

,seq,term_GO:0000003,term_GO:0000018,term_GO:0000041,term_GO:0000070,term_GO:0000075,term_GO:0000077,term_GO:0000082,term_GO:0000096,term_GO:0000118,...,term_GO:2001141,term_GO:2001233,term_GO:2001234,term_GO:2001235,term_GO:2001236,term_GO:2001242,term_GO:2001243,term_GO:2001251,term_GO:2001252,term_GO:2001257
0,MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MFYEHIILAKKGPLARIWLAAHWDKKITKAHVFETNIEKSVEGILQ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MHSTIVYACLLALAVFVALHGTPLAALAENGEGTTQPDYDNSTDYY...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MDRDAEEGRPLSLVNRRPSISAPISGRKSAPASAAAAVAAAAAAAA...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MSTIKLLIIGQLWLSIGLISGDDSLDTREGVDLVLKCRFTEHYDST...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138919,MAQYDVLIVGAGHGGAQAAVALRQNKFEGTIAIVGDEPELPYERPP...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138920,FKLLAASGLTRCGRGGLVVTETAVKIVKFHNRTFTLGPVNLKVASE...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138921,MPKGGCPKAPQQEELPLSSDMVEKQTGKKDKDKVSLTKTPKLERGD...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138922,MNKKTKLIHGGHTTDDYTGAVTTPIYQTSTYLQDDIGDLRQGYEYS...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
from sklearn.multioutput import ClassifierChain

# Preprocess the data
vectorizer = CountVectorizer(analyzer='char')
X = vectorizer.fit_transform(trainC['seq'])
y = trainC.drop('seq', axis=1).values

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,
                                                  random_state=42)

del X
gc.collect()

X_train = X_train.astype('float64')
X_val = X_val.astype('float64')

gc.collect()

# Create the LGBMClassifier model
model =  ClassifierChain(base_estimator=LGBMClassifier(objective='binary',
                                                       random_state=42),
                         random_state=42)

gc.collect()
# Train the model with early stopping
model.fit(X_train, y_train)#,
          #callbacks=[early_stopping(100), log_evaluation(100)],
          #eval_metric='multi_logloss',
          #eval_set=[(X_val, y_val)])

gc.collect()
print(model.score(X_val,y_val))

/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
/opt/conda/lib/python3.10/site-packages/lightgbm/basic.py:859: UserWarning: Converting data to scipy sparse matrix.
  _log_warning

0.0029152420370703617


In [29]:
gc.collect()

88